In [ ]:
```
22.03.21
1. pycaret(전처리X, top10) => 0.15194

22.03.23
1. dense(16-32-64-32-16-1) / elu / adam / es = 50 / rlrp = 0.2 20 / epo = 1000 / val_mae = 1.54  => 0.1454
2. dense(16-32-64-32-16-1) / elu / Nadam / es = 50 / rlrp = 0.2 45 / epo = 1000 / val_mae = 1.5246  => 0.1439 (best)


```

+ cv 나눠서 평균으로 해보기 (손동작 참고)
+ 전처리 - 이상치 제거 

In [187]:
import pandas as pd
import os
import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(mpl.__version__)
# seaborn 스타일 사용
plt.style.use('seaborn-notebook')

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


3.4.3


In [188]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [189]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [ ]:
train_X

In [ ]:
plt.figure(figsize=(12,12))
for i in range(1,9):
    plt.subplot(3,4,i)
    sns.distplot(train.iloc[:,i])
plt.tight_layout()
plt.show()

# 파이캐럿 (전처리 X)

In [ ]:
pycaret_data = pd.get_dummies(data = train, columns = ['Gender'], prefix = 'Gender')

In [ ]:
from pycaret.classification import *
from pycaret.regression import *

In [ ]:
setup_clf = setup(data = pycaret_data, target = 'Target', fold_shuffle=True, use_gpu=True, silent=True)

In [ ]:
top10 = compare_models(sort='MAE', n_select=10, fold = 5)

In [ ]:
tuned_top10 = [tune_model(i, fold = 5) for i in top10]

In [ ]:
blender_top10 = blend_models(estimator_list=tuned_top10)

In [ ]:
final_model = finalize_model(blender_top10)
prediction = predict_model(final_model, data= test)

In [ ]:
prediction

In [ ]:
prediction['Label']

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = prediction['Label']

submission

submission.to_csv('submission.csv',index=False)

# 딥러닝 기반

In [190]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from sklearn.model_selection import train_test_split

In [194]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=10, activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))            
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [195]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=45, mode='min')

In [196]:
# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

Epoch 1/1000
28/28 [==============================] - 1s 5ms/step - loss: 8.0692 - mae: 8.0692 - val_loss: 4.9444 - val_mae: 4.9444
Epoch 2/1000
28/28 [==============================] - 0s 2ms/step - loss: 2.3366 - mae: 2.3366 - val_loss: 1.9839 - val_mae: 1.9839
Epoch 3/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.7231 - mae: 1.7231 - val_loss: 1.9345 - val_mae: 1.9345
Epoch 4/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.7162 - mae: 1.7162 - val_loss: 1.9255 - val_mae: 1.9255
Epoch 5/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.7017 - mae: 1.7017 - val_loss: 1.9196 - val_mae: 1.9196
Epoch 6/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.6952 - mae: 1.6952 - val_loss: 1.9514 - val_mae: 1.9514
Epoch 7/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.6941 - mae: 1.6941 - val_loss: 1.9112 - val_mae: 1.9112
Epoch 8/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.

28/28 [==============================] - 0s 2ms/step - loss: 1.4166 - mae: 1.4166 - val_loss: 1.5810 - val_mae: 1.5810
Epoch 124/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.4149 - mae: 1.4149 - val_loss: 1.6844 - val_mae: 1.6844
Epoch 125/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4302 - mae: 1.4302 - val_loss: 1.6343 - val_mae: 1.6343
Epoch 126/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.4128 - mae: 1.4128 - val_loss: 1.5920 - val_mae: 1.5920
Epoch 127/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4001 - mae: 1.4001 - val_loss: 1.6028 - val_mae: 1.6028
Epoch 128/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.4150 - mae: 1.4150 - val_loss: 1.6624 - val_mae: 1.6624
Epoch 129/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4018 - mae: 1.4018 - val_loss: 1.6151 - val_mae: 1.6151
Epoch 130/1000
28/28 [==============================] - 0s 2ms/step - loss: 1

Epoch 184/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.3718 - mae: 1.3718 - val_loss: 1.5517 - val_mae: 1.5517
Epoch 185/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.3771 - mae: 1.3771 - val_loss: 1.5514 - val_mae: 1.5514
Epoch 186/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.3847 - mae: 1.3847 - val_loss: 1.5871 - val_mae: 1.5871
Epoch 187/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.3886 - mae: 1.3886 - val_loss: 1.5456 - val_mae: 1.5456
Epoch 188/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.3802 - mae: 1.3802 - val_loss: 1.5602 - val_mae: 1.5602
Epoch 189/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.3820 - mae: 1.3820 - val_loss: 1.5699 - val_mae: 1.5699
Epoch 190/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.3919 - mae: 1.3919 - val_loss: 1.5692 - val_mae: 1.5692
Epoch 191/1000
28/28 [==============================] - 0s 2ms

28/28 [==============================] - 0s 1ms/step - loss: 1.3664 - mae: 1.3664 - val_loss: 1.5574 - val_mae: 1.5574
Epoch 307/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.3587 - mae: 1.3587 - val_loss: 1.5624 - val_mae: 1.5624
Epoch 308/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.3720 - mae: 1.3720 - val_loss: 1.5422 - val_mae: 1.5422
Epoch 309/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.3655 - mae: 1.3655 - val_loss: 1.5557 - val_mae: 1.5557
Epoch 310/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.3700 - mae: 1.3700 - val_loss: 1.5387 - val_mae: 1.5387
Epoch 311/1000
28/28 [==============================] - 0s 1ms/step - loss: 1.3553 - mae: 1.3553 - val_loss: 1.5389 - val_mae: 1.5389
Epoch 312/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.3540 - mae: 1.3540 - val_loss: 1.5397 - val_mae: 1.5397
Epoch 313/1000
28/28 [==============================] - 0s 1ms/step - loss: 1

In [184]:
Y_prediction = model.predict(test).flatten()

In [185]:
len(Y_prediction.flatten())

2924

In [186]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = Y_prediction.flatten()

submission

submission.to_csv('submission.csv',index=False)

# 실험실

## 표준화

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
# StandardScaler객체 생성
scaler = StandardScaler()

# StandardScaler 로 데이터 셋 변환 .fit( ) 과 .transform( ) 호출
scaler.fit(train_X)
train_X_scaled = scaler.transform(train_X)

# transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
train_X_scaled = pd.DataFrame(data=train_X_scaled, columns=train_X.columns)

In [6]:
train_X_scaled

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.683382,0.632069,-0.629084,0.565272,0.150018,0.983043,0.490632,-0.659758,-0.689822,1.292920
1,-0.772731,-0.922116,-1.141679,-0.905870,-0.818704,-0.902715,-0.951302,-0.659758,1.449650,-0.773443
2,0.475366,0.832609,1.421295,0.970036,0.764655,0.652365,1.192304,-0.659758,1.449650,-0.773443
3,0.100937,-0.019686,0.908700,0.878090,0.842598,1.300315,0.680084,-0.659758,-0.689822,1.292920
4,-1.771209,-1.724275,-1.269828,-1.407577,-1.384349,-1.340640,-1.403880,-0.659758,1.449650,-0.773443
...,...,...,...,...,...,...,...,...,...,...
1248,-2.769687,-2.626705,-2.551316,-1.585474,-1.524646,-1.559602,-1.579298,-0.659758,1.449650,-0.773443
1249,-1.063954,-0.972251,-1.397977,-1.027799,-0.916690,-1.166364,-1.028486,-0.659758,1.449650,-0.773443
1250,0.017730,0.030449,-0.629084,-0.113332,0.254684,-0.160924,-0.421540,1.515707,-0.689822,-0.773443
1251,-0.647921,-0.721576,-0.757233,-0.790937,-0.696222,-0.639066,-0.846052,1.515707,-0.689822,-0.773443


# 딥러닝 기반

In [23]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from sklearn.model_selection import train_test_split

In [24]:
# train, test 데이터 분리
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=123)

In [25]:
X_train

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
525,0.620,0.470,0.145,1.0865,0.5110,0.2715,0.2565,0,0,1
229,0.375,0.275,0.095,0.2465,0.1100,0.0415,0.0775,0,1,0
690,0.615,0.505,0.190,1.4030,0.6715,0.2925,0.3650,0,0,1
303,0.660,0.520,0.190,1.5580,0.7550,0.2980,0.4000,0,0,1
289,0.600,0.465,0.165,1.0475,0.4650,0.2345,0.3150,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1238,0.425,0.340,0.120,0.3880,0.1490,0.0870,0.1250,0,0,1
1147,0.650,0.505,0.175,1.2075,0.5105,0.2620,0.3900,1,0,0
106,0.565,0.445,0.145,0.9255,0.4345,0.2120,0.2475,0,0,1
1041,0.315,0.210,0.060,0.1250,0.0600,0.0375,0.0350,0,1,0


In [131]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=10, activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))            
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='adam', 
              metrics=['mae'])

In [132]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=20, mode='min')

In [133]:
# 모델 학습
history = model.fit(X_train, Y_train, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[early_stopping_callback, checkpointer, rlrp])

Epoch 1/1000
22/22 [==============================] - 0s 5ms/step - loss: 8.3302 - mae: 8.3302 - val_loss: 6.2271 - val_mae: 6.2271
Epoch 2/1000
22/22 [==============================] - 0s 2ms/step - loss: 3.6819 - mae: 3.6819 - val_loss: 1.8998 - val_mae: 1.8998
Epoch 3/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.9440 - mae: 1.9440 - val_loss: 1.7800 - val_mae: 1.7800
Epoch 4/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.8230 - mae: 1.8230 - val_loss: 1.7451 - val_mae: 1.7451
Epoch 5/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.7987 - mae: 1.7987 - val_loss: 1.7423 - val_mae: 1.7423
Epoch 6/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.8050 - mae: 1.8050 - val_loss: 1.7490 - val_mae: 1.7490
Epoch 7/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.7970 - mae: 1.7970 - val_loss: 1.7376 - val_mae: 1.7376
Epoch 8/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.

22/22 [==============================] - 0s 2ms/step - loss: 1.4129 - mae: 1.4129 - val_loss: 1.4394 - val_mae: 1.4394
Epoch 124/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.4256 - mae: 1.4256 - val_loss: 1.4955 - val_mae: 1.4955
Epoch 125/1000
22/22 [==============================] - 0s 1ms/step - loss: 1.4461 - mae: 1.4461 - val_loss: 1.4322 - val_mae: 1.4322
Epoch 126/1000
22/22 [==============================] - 0s 1ms/step - loss: 1.4068 - mae: 1.4068 - val_loss: 1.4333 - val_mae: 1.4333
Epoch 127/1000
22/22 [==============================] - 0s 1ms/step - loss: 1.3996 - mae: 1.3996 - val_loss: 1.4296 - val_mae: 1.4296
Epoch 128/1000
22/22 [==============================] - 0s 1ms/step - loss: 1.4007 - mae: 1.4007 - val_loss: 1.4306 - val_mae: 1.4306
Epoch 129/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.4010 - mae: 1.4010 - val_loss: 1.4273 - val_mae: 1.4273
Epoch 130/1000
22/22 [==============================] - 0s 2ms/step - loss: 1

Epoch 184/1000
22/22 [==============================] - 0s 1ms/step - loss: 1.3922 - mae: 1.3922 - val_loss: 1.4257 - val_mae: 1.4257
Epoch 185/1000
22/22 [==============================] - 0s 1ms/step - loss: 1.3923 - mae: 1.3923 - val_loss: 1.4247 - val_mae: 1.4247
Epoch 186/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.3921 - mae: 1.3921 - val_loss: 1.4255 - val_mae: 1.4255
Epoch 187/1000
22/22 [==============================] - 0s 1ms/step - loss: 1.3917 - mae: 1.3917 - val_loss: 1.4251 - val_mae: 1.4251
Epoch 188/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.3920 - mae: 1.3920 - val_loss: 1.4245 - val_mae: 1.4245
Epoch 189/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.3918 - mae: 1.3918 - val_loss: 1.4250 - val_mae: 1.4250
Epoch 190/1000
22/22 [==============================] - 0s 2ms/step - loss: 1.3921 - mae: 1.3921 - val_loss: 1.4254 - val_mae: 1.4254
Epoch 191/1000
22/22 [==============================] - 0s 1ms

In [134]:
# 예측 값과 실제 값의 비교
Y_prediction = model.predict(X_test).flatten()
# flatten : 데이터 배열이 몇 차원이든 모두 1차원으로 바꿔 읽기 쉽게 해 주는 함수

# 10개 실제값과 예측값 비교
for i in range(20):
    label = Y_test.values[i]
    prediction = Y_prediction[i]
    print("실제나이: {:.3f}, 예상나이: {:.3f}".format(label, prediction))

실제나이: 4.000, 예상나이: 4.176
실제나이: 11.000, 예상나이: 9.096
실제나이: 8.000, 예상나이: 11.896
실제나이: 10.000, 예상나이: 8.639
실제나이: 5.000, 예상나이: 5.235
실제나이: 11.000, 예상나이: 11.731
실제나이: 10.000, 예상나이: 9.720
실제나이: 8.000, 예상나이: 8.011
실제나이: 7.000, 예상나이: 7.571
실제나이: 11.000, 예상나이: 9.410
실제나이: 12.000, 예상나이: 9.643
실제나이: 9.000, 예상나이: 9.907
실제나이: 5.000, 예상나이: 5.470
실제나이: 13.000, 예상나이: 12.212
실제나이: 6.000, 예상나이: 7.186
실제나이: 6.000, 예상나이: 6.731
실제나이: 6.000, 예상나이: 8.083
실제나이: 8.000, 예상나이: 8.087
실제나이: 8.000, 예상나이: 9.211
실제나이: 9.000, 예상나이: 10.414
